In [1]:
import pandas as pd
import os
from pathlib import Path

In [2]:
journey_id=1440
raw_dir=Path.cwd()/'raw data'
raw_files=os.listdir(raw_dir) #csv files for all running journeys for a day
raw_files

['20220419.csv',
 '20220420.csv',
 '20220421.csv',
 '20220422.csv',
 '20220423.csv',
 '20220424.csv',
 '20220425.csv',
 '20220426.csv',
 '20220427.csv',
 '20220428.csv',
 '20220429.csv',
 '20220430.csv',
 '20220501.csv']

In [3]:
def gen_report(file):
    df=pd.read_csv(file,
                   usecols=['Journey ID','Node ID',
                            'Campaign / Segment / Controller Name',
                            ' Channel / Segment / Controller Type',
                            ' Moved Forward',
                            'Moved Forward via Yes',
                            'Moved Forward via No',
                            'Moved Forward via Node Timeout',
                            'Total Sent',
                            'Total Viewed',
                            'Total Clicked',
                            'Errors',
                            'Goal 1 Conversions'

    ])
    
    filt=df['Journey ID']==journey_id
    df=df.loc[filt]
    df.to_csv(file,index=False)
    df.fillna(0,inplace=True)
    df['dTotal Errors'] = df.groupby(['Node ID'])['Errors'].transform('sum')
    

#     df['aSent'] = df.groupby(['Node ID'])['Unique Sent within Conversion Time'].transform('sum')

    df['aSent'] = df.groupby(['Node ID'])['Total Sent'].transform('sum')       
    df['bView'] = df.groupby(['Node ID'])['Total Viewed'].transform('sum')
    df['cClick'] = df.groupby(['Node ID'])['Total Clicked'].transform('sum')
    df.drop_duplicates(subset=['Node ID'], keep='first', inplace=True)
    new_ids=[i for i in range(1,df.shape[0]+1)]
    df['New ID']=new_ids
    df.columns=df.columns.str.strip()
    df['Moved Forward via No/Node Timeout']=df['Moved Forward via No']+df['Moved Forward via Node Timeout']


    df = df[['Node ID','New ID', 'Campaign / Segment / Controller Name',
                'Channel / Segment / Controller Type',
                'Moved Forward',
                'Moved Forward via Yes',
                'Moved Forward via No/Node Timeout',
                'Goal 1 Conversions',
                'dTotal Errors',
                'aSent',
                'bView',
                'cClick']]


    df.columns=['Node ID','New ID',
                'Campaign / Segment / Controller Name',
                'Channel / Segment / Controller Type',
                'aMoved Forward',
                'bMoved Forward via Yes',
                'cMoved Forward via No/Node Timeout',
                'goal 1 Conversions',
                'eErrors',
                'aSent',
                'bView',
                'cClick']
    filt=(df['Channel / Segment / Controller Type'] == 'Push') | (df['Channel / Segment / Controller Type'] == 'Email')
    df_push=pd.DataFrame(df.loc[filt])



    filt1=df['Channel / Segment / Controller Type'].isin(['Action','Inaction','Past behavior'])
    df_seg=pd.DataFrame(df.loc[filt1])
    df_seg=df_seg.loc[:, ['Node ID',
                          'New ID',
                          'Campaign / Segment / Controller Name',
                            'aMoved Forward',
                            'bMoved Forward via Yes',
                            'cMoved Forward via No/Node Timeout',
                            'goal 1 Conversions'

    ]]

    df_push=df_push.loc[:, ['Node ID',
                            'New ID',
                            'Campaign / Segment / Controller Name',
                            'aSent',
                            'bView',
                            'cClick',
                            'goal 1 Conversions',
                            'eErrors'

    ]]
    df_seg_melted = pd.melt(df_seg, id_vars=["Node ID",'New ID','Campaign / Segment / Controller Name'])
    df_push_melted = pd.melt(df_push, id_vars=["Node ID",'New ID','Campaign / Segment / Controller Name'])
    df_seg_melted.sort_values(by=['New ID'])
    df_push_melted.sort_values(by=['New ID'])
    final=pd.concat([df_seg_melted,df_push_melted])
    final['variable']=final['variable'].str.strip()
    final.sort_values(by=['New ID','variable'],inplace=True)
    date=str(file).split('\\')[8].split('.')[0]
    final.rename(columns={'value':date},inplace=True)
    
    
    output_path=Path.cwd()/'output'/f"{journey_id}_{date}.xlsx"
    final.to_excel(output_path,index=False)

In [4]:
for file in raw_files:
    path=raw_dir/file
    gen_report(path)

# Merging

In [7]:
#Merging
output_path=Path.cwd()/'output'
output_files=os.listdir(output_path)
output_files

['1440_20220419.xlsx',
 '1440_20220420.xlsx',
 '1440_20220421.xlsx',
 '1440_20220422.xlsx',
 '1440_20220423.xlsx',
 '1440_20220424.xlsx',
 '1440_20220425.xlsx',
 '1440_20220426.xlsx',
 '1440_20220427.xlsx',
 '1440_20220428.xlsx',
 '1440_20220429.xlsx',
 '1440_20220430.xlsx',
 '1440_20220501.xlsx']

In [8]:
mdf=pd.read_excel(Path.cwd()/"Book1.xlsx")
for file in output_files:
    df=pd.read_excel(output_path/file)
    df1=pd.merge(mdf, df, how='inner',on=["Node ID",'New ID','Campaign / Segment / Controller Name',"variable"])
    mdf=df1
mdf    

,Node ID,New ID,Campaign / Segment / Controller Name,variable,20220419,20220420,20220421,20220422,20220423,20220424,20220425,20220426,20220427,20220428,20220429,20220430,20220501
0,2,1,LTR/RPC=1,aMoved Forward,430555,299632,243959,224888,232272,187521,157141,165136,160763,146307,148724,137978,132622
1,2,1,LTR/RPC=1,bMoved Forward via Yes,430555,299632,243959,224888,232272,187521,157141,165136,160763,146307,148724,137978,132622
2,2,1,LTR/RPC=1,cMoved Forward via No/Node Timeout,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,1,LTR/RPC=1,goal 1 Conversions,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2,D0,aSent,402575,279763,227301,209836,216246,173229,144533,153468,149678,135686,138224,127368,121777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,170,44,D2,goal 1 Conversions,0,0,0,0,0,0,0,0,0,0,0,0,0
198,173,45,Ride 2 Complete?,aMoved Forward,0,0,11890,32409,39715,33700,45498,51735,50127,47276,49653,1068065,106058
199,173,45,Ride 2 Complete?,bMoved Forward via Yes,0,0,11890,32409,39715,33700,45498,51735,50127,47276,49653,42272,0
200,173,45,Ride 2 Complete?,cMoved Forward via No/Node Timeout,0,0,0,0,0,0,0,0,0,0,0,1025793,106058


In [9]:
mdf.to_excel(Path.cwd()/"merged.xlsx",index=False)